Install qiskit
and also matplotlib and pylatexenc (for nicer circuit plots)

In [ ]:
!pip install qiskit qiskit_aer qiskit_ibm_runtime matplotlib pylatexenc

### Import Qiskit

In [ ]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import QasmSimulator, StatevectorSimulator
from qiskit.visualization import plot_histogram

## Problem 1: A first quantum circuit in Qiskit

Initialise a simple circuit with 1 qubit and 1 classical bit for the output

In [ ]:
# Every quantum circuit is initialised with every qubit in |0>
qc1a = QuantumCircuit(QuantumRegister(1), ClassicalRegister(1))
# qc1a = QuantumCircuit(1,1)

In [ ]:
# In Qiskit, standard gates are methods of the circuit object


# TODO apply a single qubit gate to qubit 0

qc1a.measure(qubit=0, cbit=0)

we can also draw the circuit

In [ ]:
qc1a.draw('mpl')

Finially we run the circuit on a simulator (1000 times)
and plot the measurement results

In [ ]:
simulator = QasmSimulator()
results1a = simulator.run(qc1a, shots=1000).result()

In [ ]:
plot_histogram(results1a.get_counts())

#### Preparing Bell states

In [ ]:
# Shortcut for QuantumCircuit(QuantumRegister(2), ClassicalRegister(2))
qc1b = QuantumCircuit(2,2)


# TODO prepare a Bell state
# e.g. by doing a h gate on qubit 0 followed by a cnot on qubits 0 and 1


qc1b.measure([0,1], [0,1])
# Shortcut:
#qc1b.measure_all()

qc1b.draw('mpl')

In [ ]:
results1b = simulator.run(qc1b, shots=1000000).result()
plot_histogram(results1b.get_counts())

## Problem 2: Use different simulators in Qiskit

We are going to see how the use of different simulators affects the final result

In [ ]:
from qiskit_aer import QasmSimulator, StatevectorSimulator
from qiskit_aer.noise import NoiseModel

To use the Noise Model from real quantum devices (or run circuits on real hardware)
you can create an account on https://quantum-computing.ibm.com

On the Welcome page you can find the API Token.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', token=<IBM Quantum API key>)

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
)

Get the noise model for a real quantum device

First print the available ones

In [ ]:
service.backends(simulator=False)

Choose one and get a noise model (approximately) describing it

In [ ]:
backend = service.backend('ibm_brisbane')
noise_model = NoiseModel.from_backend(backend)
print(noise_model)

prepare the simulators

In [ ]:
statevector_simulator = StatevectorSimulator()
qasm_simulator        = QasmSimulator()
noisy_qasm_simulator  = QasmSimulator(noise_model=noise_model)

In [ ]:
qc2 = QuantumCircuit(4)


# TODO implement a circuit that prepares 1/√2 (|0000⟩ + |1111⟩)


qc2.draw('mpl')

If we use the statevector_simulator we can directly extract the coefficients in the computational basis:

In [ ]:
# Statevector simulator is the exact state at the end of the circuit, then shots=1 by default
results2 = statevector_simulator.run(qc2).result()
results2.get_statevector()

As we can see, we obtain the vector describing the state of the quantum computer.

Now we add the missing measurements of the end of the circuit

In [ ]:
qc2.measure_all()
qc2.draw('mpl')

In [ ]:
results2 = qasm_simulator.run(qc2, shots=1000).result()

plot the result of 1000 runs

In [ ]:
plot_histogram(results2.get_counts())

and finally we run it using the simulated noise model from the device we selected

In [ ]:
results2 = noisy_qasm_simulator.run(qc2, shots=1000).result()
plot_histogram(results2.get_counts())

## Problem 3: Transpile a quantum Circuit

In [ ]:
from qiskit.compiler import transpile

First we select a backend to transpile for

In [ ]:
from qiskit.providers.fake_provider import GenericBackendV2
fake_hardware_backend = GenericBackendV2(num_qubits=5)

In [ ]:
qc3 = QuantumCircuit(5)


# TODO implement a cirquit of your choice using 5 qubits

qc3.measure_all()
qc3.draw('mpl')

then we transpile the circuit for the selected backend

In [ ]:
transpiled_qc3 = transpile(qc3, backend = fake_hardware_backend)
transpiled_qc3.draw('mpl')

As you can see, transpilation greatly extend the depth of your circuit. You can use the options in the transpile function to reduce the depth of the transpiled circuit.

In [ ]:
results = fake_hardware_backend.run(transpiled_qc3, shots=1024).result()
plot_histogram(results.get_counts(transpiled_qc3))

## Problem 4:  Quantum Fourier Transform

In [ ]:
from qiskit.circuit.library import SGate, TGate
CS = SGate().control()
CT = TGate().control()

In [ ]:
def qft(qc):
    # TODO implement the QFT for 3 qubits
    pass


def initialize(qc):
    # TODO initialize to states different from |000⟩ here
    pass

In [ ]:
# HINT:

# you can use

#from qiskit.circuit.library import SGate, TGate
#CS = SGate().control()
#CT = TGate().control()
#qc.append(CS, [control,target])
#qc.append(CT, [control,target])

# or a controlled phase gate specifying the angles for S and T in terms of π

# from math import pi
# qc.cp(angle, control, target)

In [ ]:
qc4 = QuantumCircuit(3)

initialize(qc4)
qft(qc4)
qc4.measure_all()

qc4.draw(output='mpl')

In [ ]:
qc4_transpiled = transpile(qc4, simulator)
qc4_transpiled.draw(output='mpl')

In [ ]:
results4 = simulator.run(qc4_transpiled, shots=1000).result()
plot_histogram(results4.get_counts())

In [ ]:
# TODO also run the qft cirquit with a noise model like in Problem 3 and plot the results